In [16]:
import numpy as np
from pathlib import Path
import os

path = Path("spacio_training_2")

In [17]:
def pos_features(img):
    img_size = img.shape[0]

    ones = np.ones([img_size,img_size])
    v_vec = np.arange(0,img_size).reshape(img_size,1)
    h_vec = np.arange(0,img_size).reshape(1,img_size)
    v_pos = ones*v_vec
    h_pos = ones*h_vec

    return np.dstack([v_pos, h_pos])


def nearest_building_distance(img):
    # Find the distance to the nearest building from the left
    res = img * 0

    for row in range(img.shape[0]):
        e = -1
        for i, val in enumerate(img[row]):

            # Check if building in current position
            if val >= 0.001:
                e = 0
                
            # Check if building has already been seen and no building in current position
            if e != -1 and val < 0.001:
                e += 1

            res[row, i] = e

    return res

In [18]:
img = np.load(path/"processed/0_0_geom.npy")
img = img[:,:,0]

vh_pos = pos_features(img)

nearest_building_left = nearest_building_distance(img)

nearest_building_right = nearest_building_distance(np.flip(img, 1))
nearest_building_right = np.flip(nearest_building_right, 1)

nearest_building_down = nearest_building_distance(np.rot90(img, -1))
nearest_building_down = np.rot90(nearest_building_down)

nearest_building_up = nearest_building_distance(np.rot90(img))
nearest_building_up = np.rot90(nearest_building_up, -1)

feature_array = np.dstack([img, vh_pos, nearest_building_up, nearest_building_down, nearest_building_left, nearest_building_right])

feature_array.shape

(1024, 1024, 7)

In [26]:
data = os.listdir(path/"processed")
geometries = [item for item in data if "geom" in item]

for geometry in geometries:
    img = np.load(path/f"processed/{geometry}")
    img = img[:,:,0]

    vh_pos = pos_features(img)

    nearest_building_left = nearest_building_distance(img)

    nearest_building_right = nearest_building_distance(np.flip(img, 1))
    nearest_building_right = np.flip(nearest_building_right, 1)

    nearest_building_up = nearest_building_distance(np.rot90(img))
    nearest_building_up = np.rot90(nearest_building_up, -1)

    nearest_building_down = nearest_building_distance(np.rot90(img, -1))
    nearest_building_down = np.rot90(nearest_building_down)

    feature_array = np.dstack([img, vh_pos, nearest_building_up, nearest_building_down, nearest_building_left, nearest_building_right])

    np.save(path/f"processed_with_features/{geometry}", feature_array)